# Brains Behind the Bots: Classical Planning from the Ground Up 🤖🧠  
Interactive Google Colab notebook accompanying the blog post **“Brains Behind the Bots: Classical Planning from Ground Up.”**

This notebook turns the theory into code:

* Compare search algorithms (BFS vs A*).
* Generate and solve PDDL planning problems with **pyperplan**.
* Visualise exponential growth on the classic **Towers of Hanoi** benchmark.
* Leave plenty of sandboxes so you can plug in your own domains later on.

> **Tip:** Run the notebook cell‑by‑cell. Colab’s GPUs won’t speed up search, but its generous RAM will 🙂


## 0 · Setup
Install the libraries we’ll need.

In [ ]:
# ⬇️ Install (takes ~10 s on Colab Fresh Runtime)
!pip install -q pyperplan networkx matplotlib


## 1 · Search 101 on a Tiny Grid 🗺️

Before we dive into PDDL land, let’s warm up with a 2‑D grid puzzle.

* **BFS** explores level‑by‑level.  
* **A\*** uses Manhattan distance to skip blind alleys.

We’ll measure the **path length** and the **number of states expanded** (a rough proxy for runtime).

In [ ]:
import os
import random, heapq, math
from collections import deque
import matplotlib.pyplot as plt

# Build a random 15×15 grid with ~20 % walls
def make_grid(n=15, p_block=0.2, seed=0):
    random.seed(seed)
    g = [[1 if random.random() < p_block else 0 for _ in range(n)] for __ in range(n)]
    g[0][0] = g[-1][-1] = 0   # ensure start/goal are empty
    return g

GRID = make_grid()

def neighbours(pos, grid):
    x, y = pos
    for dx, dy in ((1,0),(-1,0),(0,1),(0,-1)):
        nx, ny = x+dx, y+dy
        if 0<=nx<len(grid) and 0<=ny<len(grid[0]) and grid[nx][ny]==0:
            yield (nx, ny)

def reconstruct(parent, goal):
    path = []
    node = goal
    while node is not None:
        path.append(node)
        node = parent.get(node)
    return path[::-1]

def bfs(grid, start=(0,0), goal=None):
    if goal is None: goal = (len(grid)-1, len(grid[0])-1)
    q = deque([start])
    parent = {start: None}
    while q:
        node = q.popleft()
        if node == goal:
            return reconstruct(parent, goal), len(parent)
        for nxt in neighbours(node, grid):
            if nxt not in parent:
                parent[nxt] = node
                q.append(nxt)
    return None, len(parent)

def astar(grid, start=(0,0), goal=None):
    if goal is None: goal = (len(grid)-1, len(grid[0])-1)
    h = lambda a, b=goal: abs(a[0]-b[0]) + abs(a[1]-b[1])
    open_heap = [(h(start), 0, start)]
    g_score = {start: 0}
    parent = {start: None}
    seen = 0
    while open_heap:
        f, g, node = heapq.heappop(open_heap)
        seen += 1
        if node == goal:
            return reconstruct(parent, goal), seen
        for nxt in neighbours(node, grid):
            tentative = g + 1
            if tentative < g_score.get(nxt, math.inf):
                g_score[nxt] = tentative
                heapq.heappush(open_heap, (tentative + h(nxt), tentative, nxt))
                parent[nxt] = node
    return None, seen

# Run both searches
path_bfs, expanded_bfs = bfs(GRID)
path_astar, expanded_astar = astar(GRID)

print(f"BFS   – path {len(path_bfs)} steps, explored {expanded_bfs} states")
print(f"A*    – path {len(path_astar)} steps, explored {expanded_astar} states")

# Quick scatter visual: explored states vs. strategy
plt.figure()
plt.bar(["BFS", "A*"], [expanded_bfs, expanded_astar])
plt.ylabel("States expanded")
plt.title("A* ≫ BFS on random grid (lower is better)")
plt.show()


## 2 · Planning with PDDL 🏰 (Towers of Hanoi)

We’ll feed a **domain file** + **problem file** into **pyperplan** and let it search with A\* + FF.  
Feel free to tinker with `num_disks`, `SEARCHES`, or `HEURISTICS`.

In [ ]:
from textwrap import dedent, indent

# --- Domain file -----------------------------------------------------------------
HANOI_DOMAIN = dedent("""(define (domain hanoi)
  (:requirements :typing)
  (:types disk peg)

  (:predicates
      (on      ?d - disk   ?x - (either peg disk))
      (clear   ?x - (either peg disk))
      (smaller ?d - disk   ?x - (either peg disk))
  )

  (:action move
    :parameters (?d - disk
                 ?from - (either peg disk)
                 ?to   - (either peg disk))
    :precondition (and
      (on      ?d   ?from)
      (clear   ?d)
      (clear   ?to)
      (smaller ?d   ?to))
    :effect (and
      (on    ?d   ?to)
      (clear ?from)
      (not   (on    ?d   ?from))
      (not   (clear ?to)))
  )
)
""")
with open("hanoi-domain.pddl", "w") as f:
    f.write(HANOI_DOMAIN)
print("✅ Wrote hanoi-domain.pddl")


In [ ]:
from textwrap import indent

def hanoi_problem_pddl(n_disks: int) -> str:
    """Return a PDDL problem for Towers of Hanoi with *n_disks* disks."""
    disks = [f"d{i}" for i in range(1, n_disks + 1)]

    # ─── Objects ──────────────────────────────────────────────────────────────
    obj_section = (
        "  " + " ".join(disks) + " - disk\n"
        "      pegA pegB pegC - peg"
    )

    # ─── Static 'smaller' facts ───────────────────────────────────────────────
    smaller_facts = [
        f"(smaller {disks[i]} {disks[j]})"
        for i in range(n_disks)
        for j in range(i + 1, n_disks)
    ] + [
        f"(smaller {d} {peg})"
        for d in disks
        for peg in ("pegA", "pegB", "pegC")
    ]

    # ─── Initial state ────────────────────────────────────────────────────────
    init_facts = [f"(on {disks[-1]} pegA)"]            # largest on pegA
    for i in range(n_disks - 1, 0, -1):                # d{n-1} on d{n}, …
        init_facts.append(f"(on {disks[i-1]} {disks[i]})")
    init_facts += [
        f"(clear {disks[0]})",                         # top disk clear
        "(clear pegB)",
        "(clear pegC)",
    ]

    # ─── Goal state (same stack on pegC) ──────────────────────────────────────
    goal_facts = [f"(on {disks[-1]} pegC)"] + [
        f"(on {disks[i-1]} {disks[i]})"
        for i in range(n_disks - 1, 0, -1)
    ]

    # ─── Helper to indent with *n* spaces instead of a prefix string ─────────
    ind = lambda txt, n: indent(txt, ' ' * n)

    # ─── Assemble PDDL ────────────────────────────────────────────────────────
    pddl = "\n".join([
        f"(define (problem hanoi-{n_disks})",
        "  (:domain hanoi)",
        "  (:objects",
        ind(obj_section, 6),
        "  )",
        "  (:init",
        ind("\n".join(smaller_facts), 6),
        "",
        ind("\n".join(init_facts), 6),
        "  )",
        "  (:goal (and",
        ind("\n".join(goal_facts), 6),
        "  ))",
        ")",
    ])
    return pddl


In [ ]:
# --- Make sure the domain + problem files are on disk ------------------------
def _ensure_hanoi_files(n_disks: int):
    """Write hanoi-domain.pddl and the *n_disks* problem file if they’re missing."""
    if not os.path.exists("hanoi-domain.pddl"):
        with open("hanoi-domain.pddl", "w") as f:
            f.write(HANOI_DOMAIN)                      # uses the string from the earlier cell
    prob_file = f"hanoi-problem-{n_disks}.pddl"
    if not os.path.exists(prob_file):
        with open(prob_file, "w") as f:
            f.write(hanoi_problem_pddl(n_disks))       # calls the generator we fixed

In [ ]:
from pyperplan.planner import search_plan, HEURISTICS, SEARCHES
import time, pandas as pd

def solve_hanoi(n_disks, heuristic="hff"):
    _ensure_hanoi_files(n_disks)
    problem_file = f"hanoi-problem-{n_disks}.pddl"
    t0 = time.time()
    plan = search_plan(
        "hanoi-domain.pddl",
        problem_file,
        SEARCHES["astar"],
        HEURISTICS[heuristic]
    )
    elapsed = time.time() - t0
    if plan is None:
        raise RuntimeError(f"No plan found for {n_disks} disks – check the PDDL.")
    return len(plan), elapsed


# Solve for 3 and 5 disks
for n in (3, 5):
    steps, secs = solve_hanoi(n)
    print(f"{n} disks → {steps:2d} steps, {secs:.2f} s")


In [ ]:
# Compare plan length as n grows
ns = list(range(3, 8))
results = {"disks": [], "steps": [], "seconds": []}
for n in ns:
    steps, secs = solve_hanoi(n)
    results["disks"].append(n)
    results["steps"].append(steps)
    results["seconds"].append(secs)

import pandas as pd
df = pd.DataFrame(results)
display(df)

# Plot
plt.figure()
plt.plot(df["disks"], df["steps"], marker="o")
plt.title("Towers of Hanoi – plan steps vs. number of disks")
plt.xlabel("Number of disks")
plt.ylabel("Plan length (steps)")
plt.yscale("log")
plt.grid(True)
plt.show()


### 2.1 Heuristic Shoot‑out  
`pyperplan` ships with plenty of admissible and inadmissible heuristics.  
Below we compare node counts for three favourites: **h<sub>FF</sub>**, **h<sub>add</sub>**, and **LM‑cut**.

In [ ]:
from pyperplan.heuristics.blind import h_blind
from pyperplan.heuristics.lmcut import h_lmcut
from pyperplan.heuristics.h_add import h_add

heuristics = {
    "blind": h_blind,
    "hadd": h_add,
    "lmcut": h_lmcut,
    "hff": HEURISTICS["hff"]
}

def solve_and_count(domain, problem, heuristic_fn):
    from pyperplan.search.best_first_search import best_first_search
    from pyperplan.task import read_task
    from pyperplan.heuristics.relaxation import FFHeuristic
    task = read_task(domain, problem)
    hfn = heuristic_fn(task)
    plan, stats = best_first_search(task, hfn)
    return stats["generated"], stats["expanded"], len(plan)

for name, h in heuristics.items():
    gen, exp, length = solve_and_count("hanoi-domain.pddl", "hanoi-problem-5.pddl", h)
    print(f"{name:5s}  – generated {gen:6d} / expanded {exp:6d}  → plan {length:2d} steps")


## 3 · Playground 🔬  

Feel free to:

* Swap in your own **domain/problem** pairs.
* Try different **search strategies**, e.g. `greedy`, `wastar` (weighted A\*), `ida*`.
* Build a random Blocks World instance with *N* blocks and time the heuristics.

Below is a blank cell to hack on. Happy planning!

In [ ]:
# Your experiments start here


## 4 · Wrap‑up 🎉  

Classical planning splits **thinking** (search) from **doing** (execution).  
Armed with heuristics like *FF* and abstractions like PDDL, you can tame enormous state spaces—so your bots spend more time doing and less time scratching their heads.

If you make something cool, tweet [@yourhandle] or open a PR on the blog repo. Happy building!
